* See notes in `__v1` on why we chose `fio-4k-sync-rand-write--size-per-job`

* `__v3`:
  * We only changed the fio runtime to 60s compared to `__v2`
* `v3.1`:
  * plot everything using matplotlib instead of seaborn / pandas
  * use plot layouts as we need them in the thesis


In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt
import matplotlib

from pathlib import Path
import seaborn as sns
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
savefig_enable = True
seaborn_context = "paper"
savefig_dir = "./postprocess_results"

In [ ]:
dstools.setup({
    "seaborn_context": seaborn_context,
    "savefig": {
        "enable": savefig_enable,
        "dir": Path(savefig_dir),
    }
    
})
result_storage = ResultStorage(Path("./results"))


In [ ]:
# display in this notebook
%matplotlib inline
#%matplotlib qt
#%matplotlib notebook
matplotlib.rcParams['figure.dpi'] = 120

In [ ]:
# from https://seaborn.pydata.org/generated/seaborn.plotting_context.html#seaborn.plotting_context
scalefactors = {    
    'paper': 0.8,
    'notebook': 1,
    'talk': 1.3,
    'poster': 1.6
}
def ctxfigsize(ctx, width, height):
    return tuple(scalefactors[ctx] * np.array((width, height)))

textwidth = 5.5 #inch


In [ ]:
id_vars__dottedpath_and_shortname_and_type = [
    ("subject", "test_subject", str),
    ("result.identity", "benchmark", str),
    ("result.fio_config.numjobs", "numjobs", int),
]
id_vars = [p[1] for p in id_vars__dottedpath_and_shortname_and_type]

def extract_id_var_values(output_json):
    d = output_json
    id_var_values = {}
    for dp, sn, ty in id_vars__dottedpath_and_shortname_and_type: 
        v = dotted.get(d, dp)
        if not v:
            raise Exception(f"{d['file']}: dotted path {dp} not found")
        if sn in id_var_values:
            raise Exception(f"duplicate shortname {sn}")
        try:
            id_var_values[sn] = ty(v)
        except ValueError as e:
            raise Exception(f"cannot parse v={v!r}") from e
    return id_var_values


def get_fio_write_metrics(output_json):
    d = output_json
    jobs = dotted.get(d, "fio_jsonplus.jobs")
    assert len(jobs) == 1
    j0 = jobs[0]
    jw = jobs[0]["write"]
    return jw


def to_row_dict(output_json):
    try:
        jw = get_fio_write_metrics(output_json["result"])

        return {
            **extract_id_var_values(output_json),
            
            # meta
            "file": output_json['file'],
            
            # fio
            "w_iops_mean": jw["iops_mean"],
            "w_iops_stddev": jw["iops_stddev"],
            "w_lat_mean": dotted.get(jw, "lat_ns.mean"),
            "w_lat_stddev": dotted.get(jw, "lat_ns.stddev"),
            "clat_p5": jw['clat_ns']['percentile']['5.000000'],
            "clat_p95": jw['clat_ns']['percentile']['95.000000'],
            "clat_p99": jw['clat_ns']['percentile']['99.000000'],
            "clat_p999": jw['clat_ns']['percentile']['99.900000'],
            "clat_p9999": jw['clat_ns']['percentile']['99.990000'],
        }
    except:
        print(json.dumps(output_json))
        raise
    

In [ ]:
rows = [to_row_dict(j) for j in result_storage.iter_results("motivating_fio_benchmark__v3")]
df = pd.DataFrame.from_dict(rows)
# df = df.set_index(id_vars)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Next cell is where you play around with the benchmark type

In [ ]:
df = df.query("benchmark == 'fio-4k-sync-rand-write--size-per-job'")
# df = df.query("benchmark == 'fio-4k-sync-rand-write--size-div-by-numjobs'")

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
#df = df.reset_index().drop('benchmark', axis=1).set_index(['test_subject', 'numjobs'])
df = df.drop('benchmark', axis=1)

In [ ]:
df['test_subject'] = df.test_subject.map(lambda v: "async" if v == "sync-disabled" else v)

In [ ]:
df

# Styling

In [ ]:
test_subject_order = ["devdax", "fsdax", "async", "zil-lwb", "zil-pmem"]
color = {c: sns.color_palette()[i] for i, c in enumerate(test_subject_order)}
style = dict(zip(test_subject_order, ['-', ':', '--', '-.', '-']))
marker = dict(zip(test_subject_order, ['o', '+', 'x', '^', '+']))
# print(style)

# Helpers

In [ ]:
def plt_abs_compare(subjects, value, title, unit, ylim=None, xlim=None, ax=None, data=None):
    if data is None:
        data = df.copy()
    
        data['w_lat_mean_us'] = data.w_lat_mean / 1_000
        data['w_iops_mean_k'] = data.w_iops_mean / 1_000
        
        data = data.set_index(['test_subject', 'numjobs'])

    # subjects must be ordered like test_subject_order otherwise the legend is off
    def value_list_is_sorted(l, key):
        """can't believe python doesn't have this"""
        return l == sorted(l, key=key)
    assert value_list_is_sorted(subjects, test_subject_order.index)

    if not ax:
        f = plt.figure(figsize=(0.5 * textwidth, 2))
        ax = f
    
    data = data.loc[subjects, slice(None)][value].unstack('test_subject')

    ax.set_title(title)
    if ylim:
        ax.set_ylim(ylim)
        
    ax.set_xticks(range(0, 10, 2))

    if xlim:
        ax.set_xlim(xlim)
    else:
        ax.set_xlim(0.75, 8.2)
        
    if unit:
        ax.set_ylabel(unit)
    
    for s in subjects:
        ax.plot(data[s].index, data[s], color=color[s], linestyle=style[s], marker=marker[s], label=s)
    

# ZIL-LWB Performance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(textwidth, 2), gridspec_kw={'hspace':0.4})
# "ZFS: Async vs Sync Write Performance"
plt_abs_compare(["devdax", "fsdax", "async", "zil-lwb"], "w_iops_mean_k", "Throughput [kIOPS]", None,
               ylim=(0, 1_000), ax=axes[0])
# "ZFS: Async vs Sync Write Latency"
plt_abs_compare(["devdax", "fsdax", "async", "zil-lwb"], "w_lat_mean_us", "Latency [us]", None,
               ylim=(1, 100), ax=axes[1])


handles, labels = axes[1].get_legend_handles_labels()
fig.legend(handles, labels, loc='center', bbox_to_anchor=(0.5, -0.1), ncol=4)

dstools.savefig("fio4k__lwb_iops_and_lat")

In [ ]:
fig, ax = plt.subplots(1, figsize=(0.75 * textwidth, 1.5))
plt_abs_compare(["devdax", "fsdax", "async"], "w_lat_mean_us", "Latency (zoomed) [us]", "",
               ylim=(0, 15), ax=ax)
fig.legend(loc='center', bbox_to_anchor=(0.5, -0.2), ncol=3)
ax.set_yticks(range(0 ,16, 4))
dstools.savefig("fio4k__lwb_lat_zoomed")

### Latency Numbers For Use In Text

In [ ]:
data = df.copy()
data = data.pivot_table(values="w_lat_mean", index=["numjobs", "test_subject"])
data = data.query('numjobs in [1, 4, 8]')
data = data.unstack(level=0)
# latencies
display((data / 1000).round(1))
# speedup

zil_lwb = data.query("test_subject == 'zil-lwb'")
assert len(zil_lwb) == 1
# display(zil_lwb.iloc[0])

zil_pmem = data.query("test_subject == 'zil-pmem'")
assert len(zil_pmem) == 1
# display(zil_pmem.iloc[0])

display((zil_lwb.reset_index(drop=True) / zil_pmem.reset_index(drop=True)).round(1))

# ZIL-PMEM 4k Fio Performance

## 4k write speedup in IOPS (zil-lwb as baseline)

In [ ]:
data = df.copy()
data = data.filter(["test_subject", "numjobs", "w_iops_mean", "w_iops_stddev"], axis=1)
data = data.set_index(["test_subject", "numjobs"], drop=True)
baseline = data.query("test_subject == 'zil-lwb'").droplevel(0)
print("zil-lwb")
display(baseline.sort_index())
display(data.query("test_subject == 'zil-pmem'").sort_index())

In [ ]:
# divide by baseline
speedup = data.divide(baseline, level=1)
speedup.query("test_subject == 'zil-pmem'")["w_iops_mean"].sort_index()

In [ ]:
d = speedup["w_iops_mean"].reset_index()
d = d.query("test_subject != 'devdax'")

subjects = test_subject_order.copy()
subjects.remove("devdax")
 # subjects must be ordered like test_subject_order otherwise the legend is off
def value_list_is_sorted(l, key):
    """can't believe python doesn't have this"""
    return l == sorted(l, key=key)
assert value_list_is_sorted(subjects, test_subject_order.index)
d = d[d.test_subject.isin(subjects)]


plt.figure(figsize=(textwidth, 3))
ax = plt.axes()
lp = sns.lineplot(data=d, x='numjobs', y='w_iops_mean', hue='test_subject', style='test_subject', markers=True,
                  hue_order=test_subject_order, style_order=test_subject_order, legend=False,
                  ax=ax)
lp.set_ylim((0, 12))
lp.set_title("Speedup of IOPS (Baseline: zil-lwb)", pad=16)
lp.set_ylabel("Speedup")
lp.legend(subjects, loc='lower center')


In [ ]:
speedup = speedup.w_iops_mean

In [ ]:
speedup = speedup.rename('speedup')

## Big Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(textwidth, 5), gridspec_kw=dict(hspace=0.2, wspace=0.25))

xlim=(0.8,8.2)

plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], 'speedup', 'Speedup', None,
               ax=axes[1,0], data=pd.DataFrame(speedup),
               xlim=xlim,
               )
axes[1,0].set_yticks(range(0, 20, 4))

plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], "w_iops_mean_k", "kIOPS", None,
               ylim=(1, 1_000),
               xlim=xlim,
               ax=axes[0, 0])

plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], "w_lat_mean_us", "Latency [us]", None,
#                ylim=(0, None),
               xlim=xlim,
               ax=axes[0, 1],
               )

plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], "w_lat_mean_us", "Latency zoomed [us]", None,
               ylim=(2, 25),
               xlim=xlim,
               ax=axes[1, 1])

axes[0, 0].set_xticklabels([])
axes[0, 1].set_xticklabels([])

axes[1,1].set_xlabel("numjobs")
axes[1,0].set_xlabel("numjobs")

handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc='center', bbox_to_anchor=(0.5, 0), ncol=4)

dstools.savefig("fio4k__zilpmem_results")

### We noticed that speedup varies significantly between runs for small `numjobs`, so let's investigate this by computing the Coefficient of Variation (CoV)

In [ ]:
data = df.copy()
data = data.set_index(["test_subject", "numjobs"])
data = data.loc[['zil-lwb', 'zil-pmem', 'async', 'fsdax'], ].copy()

cov = (data.w_iops_stddev / data.w_iops_mean)
data['CoV'] = cov
data['w_iops_stddev_k'] = data.w_iops_stddev / 1_000
data['w_iops_mean_k'] = data.w_iops_mean / 1_000

fig = plt.figure(figsize=(textwidth,4))
gs = fig.add_gridspec(4, 2, hspace=0.5, wspace=0.25)

ax = fig.add_subplot(gs[0:, 0])
plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], 'CoV', 'Coefficient of Variation', None,
               ax=ax, data=data,
                ylim=(0, 0.3)
               )

handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='center', bbox_to_anchor=(0.5, 0), ncol=4)

ax = fig.add_subplot(gs[0:2, 1])
plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], 'w_iops_mean_k', 'Mean [kIOPS]', None,
               ax=ax, data=data
               )
ax.set_xticklabels([])

ax = fig.add_subplot(gs[2:, 1])
plt_abs_compare(["fsdax", "async", "zil-lwb", "zil-pmem"], 'w_iops_stddev_k', 'Stddev [kIOPS]', None,
               ax=ax, data=data
               )

dstools.savefig("fio4k__coefficient_of_variation")

## Latency Percentiles

In [ ]:
data = df.copy()
data = data.set_index(['test_subject', 'numjobs'])
data = data[[f'clat_p{n9}' for n9 in ['5', '95', '99', '999', '9999']]].rename_axis("percentile", axis=1)
data.columns = data.columns.str.replace("clat_", "")
# display(data)

# data.reset_index().pivot(index=['percentile', 'numjobs'], columns='test_subject')
data = data.stack().rename("latency [us]")
data = data / 1_000
data = data.reset_index()
data

In [ ]:
tmp = data.set_index(['test_subject', 'numjobs', 'percentile']).sort_index().copy()


pMarkers = dict(zip(sorted(list(set(tmp.index.get_level_values('percentile')))), marker.values()))

def drawCol(subject, ax):
    d = tmp.loc[subject, 'latency [us]'].unstack('percentile')
    
    for col in d.columns:
        ax.plot(d.index, d[col], label=col, marker=pMarkers[col])
    ax.set_title(subject)
    ax.set_xticks(range(0, 10, 2))
    ax.set_xlim(0.8, 8.1)
    ax.set_xlabel('numjobs')
    
fig, axes = plt.subplots(1, 3, figsize=(textwidth, 1.8), gridspec_kw={'wspace': 0.4})
drawCol('zil-lwb', axes[0])
drawCol('zil-pmem', axes[1])
drawCol('async', axes[2])
handles, labels = axes[2].get_legend_handles_labels()
fig.legend(handles, labels, ncol=5, loc='center', bbox_to_anchor=(0.5, -0.2))

axes[0].set_ylim(0, 800)
axes[1].set_ylim(0, 80)
axes[2].set_ylim(0, 80)

fig.supylabel('latency [us]', x=0.02, fontsize=10)


dstools.savefig("fio4k__tail_latencies")